In [6]:
cycle_share = pd.read_csv('cycle_share.csv')

In [53]:
from __future__ import division
from __future__ import print_function

import numpy as np
import scipy as sp
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from patsy import dmatrix, dmatrices

import matplotlib as mpl
mpl.rc('font', family='NanumGothic')
mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()
pd.options.display.max_columns = 100


|Feature| |Description
|----|----|----|
|trip_id  | |
|starttime  | |이용 시작시간
|stoptime  | |이용 종료시간
|bikeid  | |
|tripduration | | 이용기간(초) 
|from_station_name | | 이용시작 정거장 
|to_station_name      | | 이용 종료 정거장
|from_station_id         | |       
|to_station_id              | |  
|usertype                      | | 유저 타입
|gender    | | 유저 성별                   
|age    | | 유저 나이
|Date         | | 이용 시작일자              
|station_id      | |               
|name               | |  이용시작 정거장          
|lat                   | |  위도      
|long                     | | 경도     
|install_date                | | 정거장 설치 날짜  
|install_dockcount              | | 정거장의 dock 갯수
|modification_date              | | 스테이션이 수정되어 위치 또는 도킹 개수가 변경된 날짜
|current_dockcount             | |  160831 기준 각 스테이션의 독 개수
|decommission_date               | | 정거장이 서비스를 종료 한 날짜
|Max_Temperature_F               | | 그 날의 최고 온도
|Mean_Temperature_F             | | 그 날의 평균 온도
|Min_TemperatureF             | | 그 날의 최저 온도
|Max_Dew_Point_F                 | | 그 날의 최고 이슬점
|MeanDew_Point_F                | | 그 날의 평균 이슬점
|Min_Dewpoint_F  | | 그 날의 최저 이슬점
|Max_Humidity       | |  그 날의 최고 습도         
|Mean_Humidity         | |  그 날의 평균 습도  
|Min_Humidity             | |   그 날의 최저 습도    
|Max_Sea_Level_Pressure_In   | |  그 날의 최고 해수면 높이  
|Mean_Sea_Level_Pressure_In     | | 그 날의 평균 해수면 높이
|Min_Sea_Level_Pressure_In       | | 그 날의 최저 해수면 높이
|Max_Visibility_Miles            | | 그 날의 최고 가시거리(mile)
|Mean_Visibility_Miles           | | 그 날의 평균 가시거리(mile)
|Min_Visibility_Miles            | | 그 날의 최저 가시거리(mile)
|Max_Wind_Speed_MPH              | | 그 날의 최고 풍속
|Mean_Wind_Speed_MPH            | | 그 날의 평균 풍속
|Max_Gust_Speed_MPH             | | 그 날의 최고 돌풍
|Precipitation_In                | | 그 날의 강수량
|Events                         | | 그 날의 날씨

In [3]:
cycle_share.head()

,Unnamed: 0,Unnamed: 0.1,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,from_station_area,day,weekend
0,0,0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,57.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
1,1,1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,47.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
2,2,2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,29.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
3,3,3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,40.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
4,4,4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,46.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1


In [23]:
cycle_share.pivot_table(values = ['install_dockcount'], index=['install_date'], columns=['station_id'])

install_dockcount                                                \
station_id               BT-01 BT-03 BT-04 BT-05 CBD-03 CBD-04 CBD-05 CBD-06   
install_date                                                                   
10/13/2014                18.0  16.0  16.0  14.0   20.0    NaN   20.0   20.0   
10/29/2015                 NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
3/18/2016                  NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
5/22/2015                  NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
6/12/2015                  NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
7/27/2015                  NaN   NaN   NaN   NaN    NaN   18.0    NaN    NaN   
7/3/2016                   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
8/9/2016                   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
9/15/2015                  NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   

                                                                            \
station_id   CBD-07 CBD-13 CD-01 CH-01 CH-02 CH-03 CH-05 CH-06 CH-07 CH-08   
install_date                                                                 
10/13/2014     20.0   18.0   NaN  16.0  18.0  16.0  16.0  16.0  18.0  28.0   
10/29/2015      NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3/18/2016       NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5/22/2015       NaN    NaN  16.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
6/12/2015       NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
7/27/2015       NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
7/3/2016        NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
8/9/2016        NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
9/15/2015       NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                                                                            \
station_id   CH-09 CH-12 CH-15 CH-16 DPD-01 DPD-03 EL-01 EL-03 EL-05 FH-01   
install_date                                                                 
10/13/2014    16.0  18.0  16.0   NaN   18.0   30.0  12.0  18.0  18.0  16.0   
10/29/2015     NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   NaN   NaN   
3/18/2016      NaN   NaN   NaN  18.0    NaN    NaN   NaN   NaN   NaN   NaN   
5/22/2015      NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   NaN   NaN   
6/12/2015      NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   NaN   NaN   
7/27/2015      NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   NaN   NaN   
7/3/2016       NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   NaN   NaN   
8/9/2016       NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   NaN   NaN   
9/15/2015      NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   NaN   NaN   

                                                                         \
station_id   FH-04 ID-04 PS-04 PS-05 SLU-01 SLU-02 SLU-04 SLU-07 SLU-15   
install_date                                                              
10/13/2014    20.0  16.0  18.0  18.0   18.0   18.0   18.0   18.0   12.0   
10/29/2015     NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   
3/18/2016      NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   
5/22/2015      NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   
6/12/2015      NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   
7/27/2015      NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   
7/3/2016       NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   
8/9/2016       NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   
9/15/2015      NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN   

                                                                           \
station_id   SLU-16 SLU-17 SLU-18 SLU-19 SLU-20 SLU-21 SLU-22 UD-01 UD-02   
install_date                                                                
10/13/2014     20.0   18.0   20.0   12.0    Na

In [24]:
cycle_share.pivot_table(values = ['current_dockcount'], index=['modification_date'], columns=['station_id'])

current_dockcount                                       \
station_id                   CBD-06 CD-01 CH-02 CH-08 DPD-03 EL-01 FH-01   
modification_date                                                          
10/29/2015                      NaN   NaN   NaN   NaN    NaN   NaN   NaN   
11/2/2015                       NaN   NaN   NaN   NaN    NaN   NaN   NaN   
11/9/2015                      18.0   NaN   NaN   NaN    NaN   NaN   NaN   
2/20/2015                       NaN   NaN   NaN   NaN    NaN   NaN   NaN   
2/23/2015                       NaN   NaN   NaN   NaN    NaN   NaN   NaN   
2/24/2015                       NaN   NaN  20.0  26.0    NaN   NaN   NaN   
3/13/2015                       NaN   NaN   NaN   NaN    NaN   NaN   NaN   
3/18/2016                       NaN   NaN   NaN   NaN    NaN   NaN   0.0   
3/24/2015                       NaN   NaN   NaN   NaN   24.0   NaN   NaN   
3/27/2015                       NaN   NaN   NaN   NaN    NaN  16.0   NaN   
3/4/2015                        NaN   NaN   NaN   NaN    NaN   NaN   NaN   
8/9/2016                        NaN   0.0   NaN   NaN    NaN   NaN   NaN   

                                                                              \
station_id        SLU-01 SLU-15 SLU-17 SLU-18 SLU-19 UW-01 UW-02 UW-04 UW-06   
modification_date                                                              
10/29/2015           NaN    NaN    NaN    NaN    NaN   0.0   NaN   NaN   NaN   
11/2/2015            NaN    NaN    NaN    0.0    NaN   NaN   NaN   NaN   NaN   
11/9/2015            NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
2/20/2015            NaN    NaN    NaN    NaN    NaN   NaN  14.0  16.0  14.0   
2/23/2015            NaN    NaN   16.0    NaN    NaN   NaN   NaN   NaN   NaN   
2/24/2015           20.0    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
3/13/2015            NaN   20.0    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
3/18/2016            NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
3/24/2015            NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
3/27/2015            NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   
3/4/2015             NaN    NaN    NaN    NaN   16.0   NaN   NaN   NaN   NaN   
8/9/2016             NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   

                         
station_id        UW-07  
modification_date        
10/29/2015          NaN  
11/2/2015           NaN  
11/9/2015           NaN  
2/20/2015          14.0  
2/23/2015           NaN  
2/24/2015           NaN  
3/13/2015           NaN  
3/18/2016           NaN  
3/24/2015           NaN  
3/27/2015           NaN  
3/4/2015            NaN  
8/9/2016            NaN

### 1. predict tripduration in member

In [51]:
cycle_share.gender.value_counts()

Male      112938
Female     30329
Other       2901
Name: gender, dtype: int64

In [54]:
le = LabelEncoder()
le.fit(cycle_share.gender)

/home/dockeruser/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


LabelEncoder()

In [55]:
le.transform(cycle_share.gender)

/home/dockeruser/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


array([2, 2, 1, ..., 0, 0, 0])

In [59]:
type(le.transform(cycle_share.gender))

numpy.ndarray

In [57]:
ohe = OneHotEncoder()
ohe.fit(le.transform(cycle_share.gender))

In [58]:
ohe.transform(le.transform(cycle_share.gender))

/home/dockeruser/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


AttributeError: 'OneHotEncoder' object has no attribute 'feature_indices_'

In [42]:
cycle_share = pd.concat([cycle_share, dmatrix('from_station_area -1', 
                                              data = cycle_share, return_type='dataframe')], 
                        axis = 1)
cycle_share.head()

,Unnamed: 0,Unnamed: 0.1,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,from_station_area,day,weekend,from_station_area[BT],from_station_area[CBD],from_station_area[CD],from_station_area[CH],from_station_area[DPD],from_station_area[EL],from_station_area[FH],from_station_area[ID],from_station_area[PS],from_station_area[SLU],from_station_area[UD],from_station_area[UW],from_station_area[WF]
0,0,0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,57.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,47.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,29.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,40.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,46.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.60595,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
cycle_share[cycle_share.usertype == 'Member'][['gender', 'age', ]]

,Unnamed: 0,Unnamed: 0.1,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,age,Date,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_Pressure_In,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,from_station_area,day,weekend
0,0,0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,57.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.605950,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
1,1,1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,47.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.605950,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
2,2,2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,29.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.605950,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
3,3,3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,40.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.605950,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
4,4,4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,46.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.605950,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
5,5,5,436,10/13/2014 10:34,10/13/2014 10:47,SEA00337,808.805,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,43.0,10/13/2014,CBD-06,2nd Ave & Spring St,47.605950,-122.335768,10/13/2014,20.0,11/9/2015,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,CBD,0,1
6,6,6,437,10/13/2014 11:35,10/13/2014 11:45,SEA00202,596.715,Occidental Park / Occidental Ave S & S Washing...,King Street Station Plaza / 2nd Ave Extension ...,PS-04,PS-05,Member,Male,39.0,10/13/2014,PS-04,Occidental Park / Occidental Ave S & S Washing...,47.600757,-122.332946,10/13/2014,18.0,NaN,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,PS,0,1
7,7,7,438,10/13/2014 11:35,10/13/2014 11:45,SEA00311,592.131,Occidental Park / Occidental Ave S & S Washing...,King Street Station Plaza / 2nd Ave Extension ...,PS-04,PS-05,Member,Male,34.0,10/13/2014,PS-04,Occidental Park / Occidental Ave S & S Washing...,47.600757,-122.332946,10/13/2014,18.0,NaN,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,PS,0,1
8,8,8,439,10/13/2014 11:35,10/13/2014 11:45,SEA00486,586.347,Occidental Park / Occidental Ave S & S Washing...,King Street Station Plaza / 2nd Ave Extension ...,PS-04,PS-05,Member,Female,43.0,10/13/2014,PS-04,Occidental Park / Occidental Ave S & S Washing...,47.600757,-122.332946,10/13/2014,18.0,NaN,18.0,NaN,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,21,0.0,Rain,PS,0,1
9,9,9,440,10/13/2014 11:35,10/13/2014 11:45,SEA00434,587.634,Occidental Park / Occidental Ave S & S Washing...,King Street Station Plaza / 2nd Ave Ext

In [8]:
cycle_share.install_date.value_counts()

10/13/2014    226215
7/27/2015       2693
6/12/2015       2134
3/18/2016       1288
10/29/2015       988
5/22/2015        958
9/15/2015        863
7/3/2016         761
8/9/2016         160
Name: install_date, dtype: int64